# Automating Frequency and Pattern Analysis: FFT and Wavelets and OOP

At first glance, frequency analysis and object-oriented programming might not seem closely related – and that’s partially true. However, we will combine these approaches by integrating automated frequency analysis, such as Fourier Transformations and Wavelet Analysis, into a class structure, allowing us to streamline calculations for our datasets.

We will begin with a brief introduction to signal analysis, which is essential for examining oceanic patterns across different frequency scales, such as ENSO and the Pacific Decadal Oscillation.

Let’s start by loading the SST data, as we did in Session 2.

Run the next cells.


In [ ]:
# import necessary libraries
import os
import requests
#import zipfile
import xarray as xr
import numpy as np
from scipy.signal import detrend
import pycwt as wavelet
from scipy.stats import chi2

# Visualization
from matplotlib import pyplot as plt
#import gsw
#import pandas as pd
import holoviews as hv
#from IPython.display import Image
import hvplot.xarray
hv.extension('bokeh')
#plt.rcParams['figure.figsize'] = (12,7)





import warnings
warnings.filterwarnings("ignore")


In [ ]:
#run the cell
#load SST data
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
ds_sst = xr.open_dataset(url,use_cftime=True)
ds_sst


We also load wind data; these are CEMS wind data that we've previously worked with. However, this time they start from the year 2000 and are gridded on a 1° x 1° grid.

**Question**: Do you remember how data can be resampled?

A:....

The wind data can be downloaded directly from the cloud: [Wind Data](https://cloud.hcu-hamburg.de/nextcloud/s/GFzRHQaZPKxmZjY), or alternatively, by running the next cell.


In [ ]:
## Define the cloud link
url = 'https://cloud.hcu-hamburg.de/nextcloud/s/GFzRHQaZPKxmZjY/download' 

## Download the file
filename = '../Data/Wind/cmems_wind_coarse.nc'
response = requests.get(url)

## Save the file locally
with open(filename, 'wb') as file:
    file.write(response.content)

## Now open the file using xarray
ds_wind = xr.open_dataset(filename, use_cftime=True)
ds_wind

# if its already stored local
#ds_wind = xr.open_dataset('../Data/Wind/cmems_wind_coarse.nc',use_cftime=True) # change path if necessary
#ds_wind


**1. Exercise:** We want to slice the SST data to match the time period of the wind data. Please perform this operation in the next cell.

In [ ]:
# your code here
ds_sst_24a = 


We need to calculate the detrended anomalies again. Do you remember how?

**2. Exercise**:  
Calculate the detrended anomalies of the SST data. We already covered this in Session 2 and partly in Session 5 . If you're unsure how to proceed, revisit the notebooks and copy the relevant code snippets.

In [5]:
# your code here

ds_sst_chunked = ....
sst_clim = ....
sst_anom = ...
sst_anom_detrended = ...

Optional: To verify that your calculations are correct, you can plot the data once.

In [ ]:
#Optional: your plot here. You can maybe plot the std of the anomalies


To identify on which timescales the SST shows significant variability, it makes sense to perform a Fourier analysis. For this, we need a time series and cannot directly work with the 3D fields. Therefore, we average the data over the Niño 3.4 region.

The Niño 3.4 region is located in the central Pacific, typically defined by the area from 5°N to 5°S latitude and from 170°W to 120°W longitude. This region is critical for monitoring and understanding the El Niño-Southern Oscillation (ENSO) phenomena. Variations in SST in this region can indicate the onset of El Niño or La Niña events, which have global climate impacts.

For further reading, you can explore the Niño 3.4 region and its significance: [Niño Regions](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni)
   

We will slice the data again for latitude and longitude, and then calculate the spatial mean to get a time series for the region.

In [7]:
#run the cell
lat_start_sst, lat_end_sst = -5, -20
lon_start_sst, lon_end_sst = 240, 280  # for 0-360
# calculate box-average 
sst_box_mean = sst_anom_detrended.sel(lon=slice(lon_start_sst, lon_end_sst), lat=slice(lat_start_sst, lat_end_sst)).mean(dim=['lon', 'lat'])


## Frequency Analysis with Fast Fourier Transform (FFT)


The Fast Fourier Transform (FFT) is a powerful tool for identifying dominant frequencies or periods within a signal, which, in our case, are the SST anomaly time series. By transforming these time series into the frequency domain, we can uncover the main cycles that characterize variability in the anomaly data.

The power spectrum, obtained by squaring the amplitude of the FFT, reveals the strength of the various frequencies present in the time series. Higher values in the power spectrum indicate a stronger presence of the corresponding frequency in the original time series. By setting a significance threshold—calculated as the mean plus two standard deviations of the power spectrum—we can highlight the most prominent frequencies, filtering them from background noise.

**Steps**:

1.We will apply the provided FFT code to our SST anomaly time series to compute and plot the power spectra.  
2.The periods that exceed the significance threshold in the power spectra will be identified.

Execute the following two code cells to begin the analysis.

In [ ]:
# run the cell
# Compute the Fast Fourier Transform (FFT)

fft_sst = np.fft.fft(sst_box_mean)

# Compute frequencies and corresponding periods for monthly data
n = len(sst_box_mean)
freq = np.fft.fftfreq(n, d=1)  # d=1 assumes monthly data points
periods = 1 / freq  # Periods will be in months

# Compute the Power Spectrum (amplitude squared)
power_sst = np.abs(fft_sst)**2

# Calculate a simple significance threshold, here set at mean + 2 * standard deviation
threshold_sst = np.mean(power_sst) + 2*np.std(power_sst)



# Plot the Power Spectrum with Periods on a logarithmic scale
valid_periods = periods[:n // 2] > 0  # Exclude the infinite period at zero frequency

plt.figure(figsize=(14, 6))

# SST Plot
plt.subplot(1, 1, 1)

plt.semilogy(periods[:n // 2][valid_periods], power_sst[:n // 2][valid_periods])  # Logarithmic scale on y-axis
plt.axhline(y=threshold_sst, color='r', linestyle='--', label='Significance Threshold')
plt.title('FFT Power Spectrum of SST Anomalies')
plt.xlabel('Period (months)')
plt.ylabel('Power (log scale)')
plt.grid(True)  # Adding grid lines
plt.legend()


plt.tight_layout()
plt.show()


In [ ]:
# run the cell
# Find the indices of significant power values
valid_periods = periods[:n // 2] > 0  # Exclude the infinite period at zero frequency
significant_sst_indices = np.where(power_sst[:n // 2][valid_periods] > threshold_sst)[0]


# Extract the corresponding significant periods
significant_sst_periods = periods[:n // 2][valid_periods][significant_sst_indices]


# Filter out infinite values from significant_sst_periods
significant_sst_periods = significant_sst_periods[np.isfinite(significant_sst_periods)]

# Print the significant periods for both time series
print("Significant SST Periods (months):", significant_sst_periods)


We identified dominant periods in our data: approximately 144 months (12 years), 96 months, 48 months, 41 months, and 18 months. These findings correspond to known climatic cycles:

- The 144-month cycle could be related to large-scale climatic phenomena. For instance, it may be linked to the the Pacific Decadal Oscillation (PDO), a multi-decadal climate variability pattern typically oscillating over periods of 20-30 years, which significantly impacts the North Pacific and broader global climate. Furthermore, it could be linked to the solar sunspot cycle, which spans around 11 years and affects solar radiation, thereby influencing global climatic patterns (for further reading, see this research on [solar cycles](https://iopscience.iop.org/article/10.1088/1748-9326/abfe8b)). Thus, the identified long-term cycles may point to the interplay between these and other slow-moving climate processes.

- The 48, 41-month cyle align closely with the El Niño-Southern Oscillation (ENSO) events, which typically have a 3 to 7-year cycle. ENSO significantly impacts sea surface temperatures, particularly in the central and eastern tropical Pacific, where the analyzed region (20°S to 5°S and 80°W to 120°W) is located. ENSO events influence global weather and climate patterns, making these cycles highly relevant to our analysis.

Now that we have identified significant periods in our data, we want to understand how these periods vary over time.

This is where Wavelet Analysis comes into play, offering a powerful method to extract both frequency and time information simultaneously. Unlike the FFT, which only provides a global frequency spectrum, wavelet analysis allows us to identify how dominant frequencies evolve over time. This is particularly useful for understanding non-stationary signals, where the frequency content changes.



## What is Wavelet Analysis?

Wavelet analysis breaks down a signal into both time and frequency components using localized wave-like functions, called wavelets. The Morlet wavelet is commonly used for climate data because it captures oscillatory patterns in the data effectively, and it is a good balance between time and frequency localization.

Wavelet analysis differs from FFT in that it analyzes the signal at different scales, allowing us to see how the strength of specific periods (or frequencies) varies over time. The FFT, in contrast, assumes the signal is stationary and provides a global view of all frequencies, but without temporal context.

**How Wavelet Analysis Works in the Code**:

In the provided wavelet code, we transform the SST anomaly time series into the wavelet domain using the Morlet wavelet. This method produces a wavelet power spectrum, which shows the strength of different periods over time. The spectrum is visualized as a contour plot, where the color intensity indicates the power of the signal at specific periods and times.

We also calculate the Cone of Influence (COI), which represents the region where edge effects from the finite length of the time series might distort the wavelet results. Values inside the COI are more reliable than those near the edges.

Additionally, we compute significance levels to determine which periods and times are statistically significant. These levels are calculated using the chi-square distribution and are plotted as contours over the wavelet power spectrum. Periods that exceed this threshold are considered statistically significant and can be interpreted with more confidence.

**<p style="color:royalblue;">Furthermore, in the notebook `wavelet_demo.ipynb`, the wavelet analysis process is visually explained step by step, providing a clearer understanding of the transformation and interpretation of the results.</p>**

By using wavelet analysis, we gain a detailed understanding of how important climatic cycles, vary in strength and timing throughout the analyzed period.

In [ ]:
# run the cell

# time array for 288 months (23 years from 2000 to 2023)
time = np.arange(288)
years = np.linspace(2000, 2023, num=288)

# Compute wavelet transform
dt = 1  # Monthly time step
mother_wavelet = wavelet.Morlet(6) # standard

sst_wavelet, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(sst_box_mean.values, dt, wavelet=mother_wavelet)

# Ensure the dimensions of the wavelet match the time series
sst_wavelet = sst_wavelet[:, :len(time)]  # Match wavelet data to the time series

# Calculate the wavelet power
power = np.abs(sst_wavelet) ** 2

# Manually calculate significance using chi-square distribution
alpha = 0.05
dof = 2  # Degrees of freedom for Morlet wavelet
signif_level = power.mean() * chi2.ppf(1 - alpha, dof) / dof
signif = power / signif_level

# Average the power and significance over time for each period
mean_power = np.mean(power, axis=1)  # Average power over time for each period
mean_significance = np.mean(signif, axis=1)  # Average significance over time for each period

# Convert frequencies to periods (in months)
periods = 1 / freqs

# Create the figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(12, 6))

# Plot the wavelet power spectrum on the left subplot
contour = ax1.contourf(years, periods, np.abs(sst_wavelet), levels=39, extend='both', cmap='jet')

# Add significance levels manually
ax1.contour(years, periods, signif, levels=[1.0], colors='white', linewidths=2)

# Plot the cone of influence (COI)
ax1.fill_between(years, coi, periods[-1], color='white', alpha=0.3)

# Set the x-axis to show years and y-axis for periods
ax1.set_xlabel('Time (years)')
ax1.set_ylabel('Period (months)')
ax1.set_yscale('log')
ax1.set_ylim(2, 144)  # Start the y-axis at 2 months to avoid displaying below 2
ax1.set_yticks([2, 6, 12, 24, 48, 96, 144, 300])
ax1.set_yticklabels(['2', '6', '12', '24', '48', '96', '144','300'])
ax1.set_title('Wavelet Spectrum of SST Box Mean Anomalies with COI')

# Add color bar to the left subplot
cbar = plt.colorbar(contour, ax=ax1, label='Wavelet Power')

# Plot the mean power and significance as a function of period on the right subplot
ax2.plot(mean_power, periods, 'b', label='Mean Power')
#ax2.plot(mean_significance, periods, 'r--', label='Mean Significance')
ax2.set_yscale('log')
ax2.set_ylim(ax1.get_ylim())  # Match the y-axis limits with the left plot
ax2.set_ylabel('Period (months)')  # Y-axis now matches the wavelet plot
ax2.set_yticks([2, 6, 12, 24, 48, 96, 144, 300])
ax2.set_yticklabels(['2', '6', '12', '24', '48', '96', '144', '300'])
ax2.set_xlabel('Power (°C²)')
ax2.set_title('Mean Power and Significance over Time')

# Add grid, legend, and format the plot
ax2.grid(True)
ax2.legend()
ax2.yaxis.set_label_position("right")
ax2.yaxis.tick_right()

# Show the full plot
plt.tight_layout()
plt.show()


Ok, this was a lot of code – once again we calculated anomalies, detrended data, and applied several analyses. Now, we want to apply the same steps to other datasets, but copying, pasting, and modifying all the code each time isn’t efficient. We’ve already made use of functions and modules, but there’s another tool available to us: **Object-Oriented Programming (OOP)** using classes.

## Using Classes for Efficient Data Processing - OOP

In OOP, we think of datasets as objects that can share common methods. This way, we can encapsulate all the repetitive tasks—like calculating anomalies, detrending, and performing frequency analysis—within a single class. Once designed, we can use this class to process any dataset without redoing the code for each new case.

### Why Use Classes?

By organizing our code into a class, we not only reduce redundancy but also make our code more scalable, reusable, and modular. If we want to perform similar calculations for another dataset (e.g., wind data), we don’t have to rewrite any of the functions – we just call the appropriate methods from the class. This also helps to ensure consistency in how we handle and analyze different datasets.

For this purpose, I’ve designed the `TimeSeriesAnalyzer` class in the Python script `timeseries_analyzer.py`  (located in the Modules folder – feel free to take a brief look).  

<p style="color:royalblue;">Note that this is a simplified introduction to Object-Oriented Programming (OOP). In practice, we often work with more sophisticated classes, such as those provided by <code>xarray</code>, which we've been using throughout the course to efficiently handle multidimensional climate data.</p>



In [11]:
# run the cell
# append the Modules folder to path -> we have done it with the ekman_properties.py module before
import sys
sys.path.append('../Modules')  

# imoort
from timeseries_analyzer import TimeSeriesAnalyzer 

Print the docstring of the `TimeSeriesAnalyzer` Class to get a quick overview:

In [ ]:
# your code here to get the docstring of the class


#### Why Classes are Helpful:

**Efficiency**: Instead of running individual scripts for different datasets, the class provides a unified framework that can process any dataset with the same structure (daily or monthly resoulution, 3 (time, lat, lon) or 4 dimensions (time, depth, lat, lon)).  
    
**Modularity**: Methods are self-contained, so debugging and extending functionality is much simpler.
    
**Reusability**: The class can be reused across different datasets, whether it’s SST, wind data, or other climate variables.

**Scalability**: As the analysis grows more complex, we can continue adding methods and functionality to the class without needing to rewrite existing code.

Applying the Class to Your Data

Let’s now quickly reproduce the calculations we have done before, but this time using the TimeSeriesAnalyzer class to automate the process. Afterward, you’ll get a chance to do the same for the wind data as a major task.

In [13]:
# run the cell
# Create an instance of TimeSeriesAnalyzer with the SST dataset
processor = TimeSeriesAnalyzer(ds_sst24a)

# Compute anomalies
sst_anomalies = processor.compute_anomalies_and_detrend()
print("Anomalies:\n", sst_anomalies)


With `processor.plot_std_and_annual_var('sst', vmax=5)`, we calculate and plot the standard deviation (variability) of the SST data, along with the annual variability. This gives us a quick visual representation of how the data varies over time and across seasons. The `vmax=5` parameter sets the maximum value of the colorbar, controlling the upper bound for the variability being displayed in the plot. This helps in visually interpreting the intensity of variations across the dataset.

Run the next cell


In [ ]:
# run the cell
# calculate and plot variability and annual variability
processor.plot_std_and_annual_var('sst', vmax=5)

With `processor.filter_and_analyze_cycle('sst')`, we apply both high-pass and low-pass filters to the monthly anomalies of SST. The function also plots the variability of the monthly anomalies, as well as the variability after applying the low-pass and high-pass filters. This helps us assess the presence of variability across different timescales in the data, allowing us to better understand on which temporal scales the anomalies exhibit significant variations.

Run the next cell.

In [ ]:
# run the cell
# Perform filtering and analyze cycles
processor.filter_and_analyze_cycle('sst')


With `processor.compute_fft(variable='sst', lon_min=240, lon_max=280, lat_min=-5, lat_max=-20)`, we compute the Fast Fourier Transform (FFT) of the SST anomalies for the specified spatial region. This process helps us identify the dominant periods in the time series, revealing the most significant cycles present in the SST data over time.

In [ ]:
#run the cell
#Perform FFT analysis
processor.compute_fft(variable='sst', lon_min=240, lon_max=280, lat_min=-5, lat_max=-20)


With `processor.wavelet_analysis(variable='sst', lon_min=240, lon_max=280, lat_min=-5, lat_max=-20)`, we perform a wavelet analysis to examine how the significant frequencies and periods change over time. This method allows us to explore time-localized variations in the SST anomalies, giving us a deeper insight into the temporal evolution of different cycles.

Run the next cell.

In [ ]:
# run the cell
# wavelet analysis
processor.wavelet_analysis(variable='sst', lon_min=240, lon_max=280, lat_min=-5, lat_max=-20)

**3. Exercise**

Now it’s your turn to adapt this analysis for the wind data. Before you begin, you need to calculate the Ekman properties (e.g., Ekman transport, Ekman pumping). Do you remember how? Once you’ve calculated these, save them as new variables within your wind dataset.

Your task:

1. Calculate Ekman properties and store them as new variables within your `ds_wind`. Apply a mask using the variable `number_of_observations`    
2. Create an instance of the TimeSeriesAnalyzer class for the wind data.  
3. Follow the same analysis steps that we performed for the SST data:
    - compute anomalies  
    - Compute and plot the variability and annual variability.  
    - Filter the anomalies using a low-pass and high-pass filter, and plot the variability on these different scales.  
    - Perform an FFT analysis to determine the dominant periods in the wind data. Choose the same region as SST. Be careful to check the lat and lon dimensions of your wind dataset before starting, as they may differ from the SST data. Adjust the slicing accordingly to match the same region.  
    - Conduct a wavelet analysis to investigate the temporal evolution of these dominant periods.
Finally, compare the dominant frequencies of the SST and the vertical Ekman velocity to see how they align or differ.

Good luck!


In [18]:
# run the cell

from ekman_dynamics import compute_ekman_properties

In [19]:
# your code here to calculate Ekman properties and store them in ds_wind


In [20]:
# your code here to apply a mask using number_of_observations

#'number_of_observations' is the variable used as a mask
mask = ...

# Apply the mask to the eastward wind data
ds_wind = ...

In [ ]:
# Optional: plot the temporal mean of a variable to check if everything is correct


In [22]:
# your code here
# Create an instance of TimeSeriesAnalyzer with the Wind dataset
processor_wind = ...


# Compute anomalies
wind_anomalies = ...
#print("Anomalies:\n", anomalies)

In [ ]:
# your code here to calculate and plot std and annual std



In [ ]:
# your code here
# Perform filtering and analyze cycles


In [ ]:
# your code here: Tip: check lat,lon carefully
#Perform FFT analysis


In [ ]:
# your code here: Tip: check lat,lon carefully
# wavelet analysis


Compare the power spectra of SST and vertical Ekman velocity from both the FFT and wavelet analyses. Discuss the dominant periods you observe in each dataset, and how they correspond to one another or not. 

Summarize your findings, highlighting any similarities or differences in the frequency and temporal patterns between the two datasets.



Your Observations: 

....


In [30]:
#save files

processor.save_results('../Data/SST/SST_Anomalies.nc')
processor_wind.save_results('../Data/Wind/Wind_Anomalies.nc')


In [31]:
#close files

#ds_sst.close()
#ds_wind.close()

Through the FFT and wavelet analyses, we have uncovered the dominant periods and temporal variability in both SST and vertical Ekman velocity data. This exploration allows us to understand how these variables fluctuate across different timescales. By investigating variability on various frequencies, such as interannual (e.g., ENSO) or decadal (e.g., PDO) scales, we can better comprehend the underlying dynamics driving changes in ocean patterns.

The comparison of SST and vertical Ekman velocity highlights how these phenomena are interconnected, with similar dominant periods reflecting common external forcings like wind patterns and ocean-atmosphere interactions. Understanding these relationships is crucial for interpreting variability in climate systems and their impacts on oceanographic processes.

In addition to the insights we've gained from our analyses of SST and vertical Ekman velocity, it's important to recognize that any analysis of variability, frequencies, or patterns is highly dependent on the dataset used—especially the length of the time series. For example, a short time series of just one year would not reveal interannual variability, while datasets shorter than 10 years would struggle to capture decadal fluctuations.

Furthermore, the prominence of certain signals can vary depending on the specific time period analyzed. For instance, a wavelet analysis conducted for the years 1900–1920 might reveal different patterns compared to an analysis for 2000–2020, highlighting that ocean-atmosphere variability evolves over time. This makes it crucial to consider both the temporal coverage and the length of the dataset when interpreting the results of frequency analyses.

**Extra Exercise** 

Perform the analysis using the longer SST dataset, which spans from 1854 to 2024. You are encouraged to explore various time periods and lengths of the time series. Investigate how different temporal windows affect the identified dominant frequencies, variabilities, and patterns. This will help you understand the impact of dataset length and coverage on your frequency and variability analyses.


## EOF Analysis

For a more comprehensive understanding of such variability, longer datasets and a careful examination of different time windows are essential, as they allow us to uncover patterns and signals that might otherwise be missed in shorter or less representative datasets.In addition to the analyses we performed, further investigations can be carried out using advanced techniques like **Empirical Orthogonal Function (EOF) analysis**. EOF analysis is a statistical method used to identify dominant modes of variability in spatiotemporal data. It helps in simplifying the complex relationships within the data by decomposing the dataset into principal components.

For a detailed example of how EOF analysis is applied, refer to the accompanying notebook `EOFanalysis_demo.ipynb`. This analysis can provide deeper insights into the spatial patterns of variability, complementing the frequency-based methods we've already explored.